# initialization

In [1]:
%env PYSPARK_SUBMIT_ARGS=--driver-java-options -Dio.netty.tryReflectionSetAccessible=true --conf spark.executor.extraJavaOptions=-Dio.netty.tryReflectionSetAccessible=true pyspark-shell

env: PYSPARK_SUBMIT_ARGS=--driver-java-options -Dio.netty.tryReflectionSetAccessible=true --conf spark.executor.extraJavaOptions=-Dio.netty.tryReflectionSetAccessible=true pyspark-shell


In [2]:
# Local path to gluten jar.
gluten_target_jar='/opt/gluten/jars/gluten-velox-bundle-spark3.5_2.12-linux_amd64-1.5.0-SNAPSHOT.jar'

# Select workload. Can be either 'tpch' or 'tpcds'.
workload='tpch'

# Run with gluten. If False, run Spark.
run_gluten=True

# TPC tables
#tpch_tabledir='/opt/spark/database/tpch_sf10_parquet_zstd'
#tpcds_tabledir='/opt/spark/database/tpch_sf10_parquet_zstd'

tabledir = '/opt/spark/database/tpch_sf100_parquet_zstd'

# TPC queries
#tpch_query_path='/opt/spark/tpch-queries'
#tpcds_query_path='/opt/spark/tpcds-queries'

tpc_query_path = '/opt/spark/tpch-queries/'

# Parallelism
executors_per_node=1

#gluten_tpch_task_per_core=2
#gluten_tpcds_task_per_core=4
#spark_tpch_task_per_core=8
#spark_tpcds_task_per_core=8

task_per_core=2

# Offheap ratio. 0 to disable offheap for Spark.
# onheap:offheap = 1:2
#spark_offheap_ratio=2.0
# onheap:offheap = 1:7
#gluten_offheap_ratio=7.0

offheap_ratio = 7.0

# spark.io.compression.codec
spark_codec='lz4'
# spark.gluten.sql.columnar.shuffle.codec
gluten_codec='lz4'

In [3]:
%run ./native_sql_initialize.ipynb

home: /home/spark
cwd: /opt/spark/work-dir/ipython


,Environment Variable,Value
0,HOSTNAME,ip-10-167-100-116.ec2.internal
1,PWD,/opt/spark/work-dir
2,HOME,/home/spark
3,TERM,xterm-color
4,SHLVL,0
5,SPARK_HOME,/opt/spark
6,PATH,/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:...
7,_,/usr/local/bin/jupyter
8,LC_CTYPE,C.UTF-8
9,JPY_PARENT_PID,211


localhost: ip-10-167-100-116.ec2.internal
ip: 10.167.100.116
Spark version from SPARK_HOME: 3.5.2


# Application Level Configuration

In [4]:
if run_gluten:
    sct=GlutenSparkContext(executors_per_node, task_per_core, gluten_target_jar, offheap_ratio)
    sct.conf.set('spark.gluten.sql.columnar.shuffle.codec', gluten_codec)
else:
    sct=VanillaSparkContext(executors_per_node, task_per_core, gluten_target_jar, offheap_ratio)
    sct.conf.set('spark.io.compression.codec', spark_codec)

In [5]:
if workload.lower()=="tpch":
    bm=TPCHBenchmark(sct, tabledir, 'parquet', tpc_query_path)
else:
    bm=TPCDSBenchmark(sct, tabledir, 'parquet', tpc_query_path)
bm.initialize()


            executors per node: 1
            parallelism: 16
            executor memory: 3807m
            offheap memory: 26650m
        
spark.serializer:  org.apache.spark.serializer.KryoSerializer
master:  spark://ip-10-167-100-116.ec2.internal:7077


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/06 05:58:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/06 05:58:55 WARN VeloxListenerApi: Memory overhead is set to 2048MiB which is smaller than the recommended size 7995MiB. This may cause OOM.
W20250506 05:58:56.629297  1038 MemoryArbitrator.cpp:84] Query memory capacity[1.50GB] is set for NOOP arbitrator which has no capacity enforcement
/opt/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


appid:  app-20250506055856-0001
SparkConf:


,key,value
0,spark.eventLog.enabled,true
1,spark.executor.memoryOverhead,2048m
2,spark.gluten.sql.columnar.shuffle.codec,lz4
3,spark.sql.hive.metastore.jars,maven
4,spark.sql.shuffle.partitions,16
5,spark.executor.extraJavaOptions,-Djava.net.preferIPv6Addresses=false -XX:+Igno...
6,spark.cleaner.periodicGC.interval,10s
7,spark.serializer,org.apache.spark.serializer.KryoSerializer
8,spark.app.submitTime,1746511134830
9,spark.eventLog.dir,/opt/spark/events


start run:  app-20250506055856-0001
http://10.167.100.116:18080/history/app-20250506055856-0001/jobs/


# Run Workload

In [6]:
bm.test_tpc.power_run(explain=False, print_result=False, load_table=True, action=lambda df: df.collect())

Loading all tables: ['customer', 'lineitem', 'nation', 'orders', 'part', 'partsupp', 'region', 'supplier']


start query q1, application id app-20250506055856-0001
Start time : 18478.817360843


start query q2, application id app-20250506055856-0001
Start time : 18501.717931269


start query q3, application id app-20250506055856-0001
Start time : 18512.545979805


start query q4, application id app-20250506055856-0001
Start time : 18534.952470136


...

start query q18, application id app-20250506055856-0001
Start time : 19148.156279809


start query q19, application id app-20250506055856-0001
Start time : 19260.013995246


start query q20, application id app-20250506055856-0001
Start time : 19268.627049336


start query q21, application id app-20250506055856-0001
Start time : 19281.96264492


start query q22, application id app-20250506055856-0001
Start time : 19563.381574946


In [7]:
bm.test_tpc.print_result()

{'q1': {'query_name': 'q1', 'application_id': 'app-20250506055856-0001', 'application_time_taken': 22.824918061000062, 'query_status': 'pass', 'start_time': 1746511209.859102}, 'q2': {'query_name': 'q2', 'application_id': 'app-20250506055856-0001', 'application_time_taken': 10.801454642001772, 'query_status': 'pass', 'start_time': 1746511232.7596726}, 'q3': {'query_name': 'q3', 'application_id': 'app-20250506055856-0001', 'application_time_taken': 22.370196338000824, 'query_status': 'pass', 'start_time': 1746511243.5877209}, 'q4': {'query_name': 'q4', 'application_id': 'app-20250506055856-0001', 'application_time_taken': 18.054631803999655, 'query_status': 'pass', 'start_time': 1746511265.9942107}, 'q5': {'query_name': 'q5', 'application_id': 'app-20250506055856-0001', 'application_time_taken': 78.1411037939979, 'query_status': 'pass', 'start_time': 1746511284.0701957}, 'q6': {'query_name': 'q6', 'application_id': 'app-20250506055856-0001', 'application_time_taken': 4.764243353998609, 

In [8]:
bm.collect_profile()

appid is  app-20250506055856-0001
saved to /opt/spark/work-dir/profile/


In [10]:
import gzip
with open(f'/opt/spark/work-dir/profile/{bm.appid}/{bm.appid}', 'rb') as f_in, \
    gzip.open(f'/opt/spark/work-dir/profile/{bm.appid}/{bm.appid}.gz', 'wb') as f_out:
    f_out.writelines(f_in)

FileNotFoundError: [Errno 2] No such file or directory: '/opt/spark/work-dir/profile/app-20250506055856-0001/app-20250506055856-0001'